In [1]:
import os
import joblib

# Folder to store uploaded files 


# Load models
#vectorizer, dbscan_model = load_models()



def handle_upload(change):
    uploaded_files_path = UPLOAD_FOLDER
    
    if not os.path.exists(uploaded_files_path):
        os.mkdir(uploaded_files_path)
    
    for file in change['new']:
        filename = change['new'][file]['metadata']['name']
        unique_filename = str(uuid.uuid4()) + "-" + filename
        
        file_path = os.path.join(uploaded_files_path, unique_filename)
       
        with open(file_path, "wb") as fp:
            fp.write(change['new'][file]['content'])
            
        print(f"{filename} saved to {file_path}")




In [3]:
len(paragraph_data)

45

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin

import utils
from predictors import predict_paragraphs, predict_title, segment_documents_into_paragraphs
from preprocessing import text_normalization, postprocess_titles
from config import UPLOAD_FOLDER
class ExtractAndPredict(BaseEstimator, TransformerMixin):


    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        text = utils.process_folder(X)
        # Concatenate text into one long string
        text = "".join(text.values())  
        #paragraphs = predict_paragraphs(text, self.dbscan, self.vectorizer)
        #print(type(text))
        paragraphs = segment_documents_into_paragraphs([text], eps=0.5, min_samples=2, min_paragraph_size=3, max_paragraph_size=10)
        return paragraphs

class CleanParagraphs(BaseEstimator, TransformerMixin):
    
    def fit(self, X, y=None):
        return self
    def transform(self, paragraphs):
        #clean = [text_normalization(p) for p in paragraphs] 
        return paragraphs

class TitleGenerator(BaseEstimator, TransformerMixin):

    def __init__(self):
        self.tokenizer, self.model = utils.load_title_generator()
        self.orig_paragraphs = []
    
    def fit(self, X, y=None):
        self.orig_paragraphs = X
        return self
    
    def transform(self, clean_paragraphs):
        results = []
        for idx, para in enumerate(clean_paragraphs):
            title = predict_title(para, self.tokenizer, self.model) 
            title = postprocess_titles(title)
            orig = self.orig_paragraphs[idx]
            results.append((idx, orig, title))

        return results


pipe = Pipeline([
    ('extract', ExtractAndPredict()),
    ('clean', CleanParagraphs()),
    ('title', TitleGenerator())
])

paragraph_data = pipe.fit_transform(UPLOAD_FOLDER)
#utils.save_paragraphs(paragraph_data)

In [7]:
paragraph_data

[(0,
  'Markov Chains\nIntroduction\nMarkov chains and Monte Carlo Markov chains are mathematical processes that are based on probability and are commonly used in statistics. A Markov chain is a random process that moves from one state to another, with the next state depending only on the current state - hence it has the Markov "memoryless" property. A simple example is modeling a sequence of days with sunny or rainy weather, where the weather on a given day is randomly determined based solely on the weather of the previous day. Monte Carlo Markov chains are a class of algorithms that rely on repeated random sampling to obtain numerical results, leveraging the Markov chain framework to model systems like weather, stock prices, or molecular interactions. A key application is statistical inference - using Monte Carlo simulation of a Markov process to estimate properties of complex distributions and models that cannot be easily analyzed directly. Overall, Markov chains provide a mathemati

In [8]:
len(paragraph_datah_data)

10

In [3]:
from sentence_transformers import SentenceTransformer, util
import torch


In [4]:
model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
type(paragraph_data)

In [9]:
import joblib
process_para_pipeline = joblib.dump(pipe,"models/process_para_pipeline.pkl")

In [12]:
import dominate
from dominate.tags import div, p, style, h1

def convert_to_html(results):
    doc = dominate.document(title='Processed Document')

    STYLE = """
    body {
        font-family: 'Georgia', serif;
        margin: 0;
        padding: 0;
        background: #fff;
        color: #333;
        line-height: 1.6;
    }
    .container {
        width: 80%;
        max-width: 800px;
        margin: 0 auto;
        padding: 20px;
    }
    h1.title {
        text-align: center;
        font-size: 28px;
        margin-top: 50px;
        margin-bottom: 50px;
    }
    .subtitle {
        font-weight: bold;
        font-size: 20px;
        margin-top: 30px;
        margin-bottom: 10px;
    }
    .paragraph {
        font-size: 18px;
        margin-bottom: 20px;
        text-align: justify;
        text-indent: 40px;
    }
    """

    with doc.head:
        style(STYLE)

    with doc:
        with div(cls='container'):
            h1('Processed Document', cls='title')
            for idx, content, title in results:
                with div().add(p(f"{idx+1}. {title}", cls='subtitle')):
                    p(content, cls='paragraph')

    html_file = open('output.html','w')
    html_file.write(doc.render())
    return doc


In [13]:
convert_to_html(paragraph_data)

<dominate.document "Processed Document">

In [4]:
import pandas as pd
def save_csv(paragraphs):
    #df = pd.DataFrame(paragraphs, orient='index', columns=['para_content','para_title'])
    #df.index.name = 'para_idx'
    #df.reset_index(inplace=True)
    df.to_csv('gen_para.csv', header=True, index=False)

In [5]:
df = pd.DataFrame(paragraph_data,columns=['para_idx','para_content','para_title'])

In [6]:
df

,para_idx,para_content,para_title
0,0,Markov Chains\nIntroduction\nMarkov chains and...,Markov Chains Monte Carlo
1,1,A Markov chain is a random process that moves ...,"Markov chain random process moves state , depe..."
2,2,A simple example is modeling a sequence of day...,simple example modeling sequence days sunny ra...
3,3,"Based on P, foundational Markov chain analysis...",Foundational Markov chain analysis provides eq...
4,4,\n\nText Generation through Markov Processes\n...,Text Generation Markov Processes
5,5,The Markov assumption reduces complexity for m...,Markov assumption reduces complexity model lea...


In [7]:
type(df)

pandas.core.frame.DataFrame

In [8]:
save_csv(df)

In [ ]:
df = pd.read_csv('gen_para.csv')

In [ ]:
df

# Pipeline for Search

In [ ]:
import spacy
from sentence_transformers import SentenceTransformer, util
import torch
def search_paragraphs(query, paragraphs):
    nlp = spacy.load("en_core_web_sm")

    # Load Sentence Transformer model
    model_name = "all-mpnet-base-v2"
    sentence_transformer = SentenceTransformer(model_name)

    # Extract content from the list of tuples
    paragraph_contents = [title+ " " + content for idx, content, title in paragraphs.values]

    query_embedding = sentence_transformer.encode([query])[0]

    # Calculate paragraph embeddings
    paragraph_embeddings = sentence_transformer.encode(paragraph_contents)

    # Calculate cosine similarity
    scores = util.pytorch_cos_sim(torch.tensor([query_embedding]), torch.tensor(paragraph_embeddings))[0]
    scores = scores.cpu().numpy()

    # Get the indices of top results
    top_indices = scores.argsort()[::-1][:5]

    # Create a list of tuples with score, index, and content
    top_results = [(scores[idx], idx, paragraphs.iloc[idx][2], paragraphs.iloc[idx][1]) for idx in top_indices]

    return top_results

In [ ]:
# Search query
user_query = """A casino"""

# Call the search function
result = search_paragraphs(user_query, df)


In [20]:
df.iloc[4]['para_content']

'\n\nText Generation through Markov Processes\nText generation can be formulated as a sequential decision-making problem well-suited for Markov decision processes (MDPs). Such statistical text generation circumvents manual rule-encoding. \nMarkov decision processes apply such iterative reward feedback grounded in state transitions to optimize text generation policies. By rewarding linguistic coherency, Markovian reinforcement learning promises to advance automated and creative text generation.'

In [21]:
result

[(0.12053793,
  2,
  'simple example modeling sequence days sunny rainy weather , given day randomly determined based solely previous .',
  'A simple example is modeling a sequence of days with sunny or rainy weather, where the weather on a given day is randomly determined based solely on the weather of the previous day. This pure probabilistic approach enables modeling phenomena from physics, biology, and beyond for theoretical and practical insights. This establishes a lightweight, yet versatile approach compared to heavy parameterization with neural networks. By optimizing cumulative future reward, the model learns probable transitions between words reflecting sensible narratives. \nFor example, initializing the state with "Alice was" and sampling subsequent actions as words yield:\n"Alice was heading to the store when..."\n"Alice was shocked to discover that..."\n"Alice was running late for her appointment..."\nThe model associates those sentences starting with the phrase "Alice wa

In [18]:
import pathlib
import subprocess
import pkg_resources

# Use pip freeze to generate requirements from current env
result = subprocess.run(['pip', 'freeze'], stdout=subprocess.PIPE)
requirements = result.stdout.decode()

In [19]:
requirements

'absl-py==1.0.0\naccelerate==0.25.0\naiohttp==3.9.1\naiosignal==1.3.1\naltair==5.2.0\nargon2-cffi==21.3.0\nargon2-cffi-bindings==21.2.0\nasttokens==2.0.5\nastunparse==1.6.3\nasync-timeout==4.0.3\nattrs==21.4.0\nbackcall==0.2.0\nbackports.zoneinfo==0.2.1\nbeautifulsoup4==4.12.2\nblack==22.1.0\nbleach==4.1.0\nblinker==1.7.0\nblis==0.7.11\nbreadability==0.1.20\ncachetools==5.0.0\ncatalogue==2.0.10\ncertifi==2019.11.28\ncffi==1.15.0\nchardet==3.0.4\ncharset-normalizer==3.3.2\nclick==8.0.3\ncycler==0.11.0\ncymem==2.0.8\ndatasets==2.15.0\ndbus-python==1.2.16\ndebugpy==1.5.1\ndecorator==5.1.1\ndefusedxml==0.7.1\ndill==0.3.7\ndocopt==0.6.2\ndocx==0.2.4\ndocx2txt==0.8\ndominate==2.9.0\nen-core-web-sm @ https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.2.0/en_core_web_sm-3.2.0-py3-none-any.whl\nentrypoints==0.3\nexecuting==0.8.2\nfilelock==3.13.1\nflatbuffers==2.0\nfonttools==4.29.1\nfrozenlist==1.4.1\nfsspec==2023.10.0\ngast==0.5.3\ngensim==4.3.2\ngitdb==4.0.11\nGitPy

In [20]:
# Define the string


# Open the file in write mode
with open("req.txt", "w") as text_file:
    # Write the string to the file
    text_file.write(requirements)

# Close the file
text_file.close()
